<a href="https://colab.research.google.com/github/worldterminator/consolidation/blob/main/hospitaldata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

assuming the compendium has the latest CCN by the end of that year (i.e. that if the hospital went through acquisition/merger or consolidation, the compendium captured the last event)



*   this demonstration used CMS CHOW data updated for Q4 2022 and compendium 2022. note that the CMS data includes all documented change since 2016—I thought the easiest way is to track CCN changes of hospitals that went through acquisition/merger or consolidation, and match the lastest CCN to the compendium (by the end of our time frame).
*   whether we'd account for how long a provider has been enrolled. the CMS datasets has enrollment ID but each provider could have multiple if they applied repeatedly in different circumstances. technically it's possible though



In [ ]:
import pandas as pd
chow_data = pd.read_csv('/chowq42022.csv', encoding='ISO-8859-1')

# convert 'EFFECTIVE DATE' to DateTime format
chow_data['EFFECTIVE DATE'] = pd.to_datetime(chow_data['EFFECTIVE DATE'], errors='coerce')
# extract the year from the 'EFFECTIVE DATE' in case we'll add more years
chow_data['Year'] = chow_data['EFFECTIVE DATE'].dt.year


In [ ]:
# check manually, real quick
consolidation_rows = chow_data[chow_data['CHOW TYPE TEXT'] == 'Consolidation']
# use the names I found from CHOW dataset to check new CCNs in the compendium
print(consolidation_rows[['ORGANIZATION NAME - SELLER', 'ORGANIZATION NAME - BUYER']])

Empty DataFrame
Columns: [ORGANIZATION NAME - SELLER, ORGANIZATION NAME - BUYER]
Index: []


In [ ]:
unique_values = chow_data['CHOW TYPE TEXT'].unique()
print(unique_values)

['ACQUISITION/MERGER' 'CHANGE OF OWNERSHIP']


In [ ]:
distribution=chow_data['CHOW TYPE TEXT'].value_counts()
print(distribution)

CHOW TYPE TEXT
CHANGE OF OWNERSHIP    413
ACQUISITION/MERGER      79
Name: count, dtype: int64


In [ ]:
# consider 'CCN - SELLER' as old CCN and 'CCN - BUYER' as new CCN for certain event types
# CMS explained that in case of acquistion/merger, only the buyer's identification numer remains
ccn_changes = chow_data[chow_data['CHOW TYPE TEXT'].isin(['ACQUISITION/MERGER'])]
ccn_mapping = dict(zip(ccn_changes['CCN - SELLER'], ccn_changes['CCN - BUYER']))
# function to recursively find the final CCN
def get_final_ccn(ccn):
    path = set()  # to avoid loops
    while ccn in ccn_mapping:
        if ccn in path:
            break
        path.add(ccn)
        ccn = ccn_mapping[ccn]
    return ccn


# apply the final i.e. by the year of 2022 CCN resolution to the CHOW dataset
# so those sold in acquisition/merger are assigned the latest one and can be identified
chow_data['FINAL CCN'] = chow_data['CCN - SELLER'].apply(get_final_ccn)
# replacing *seller* CCNs with the most updated CCN by Q4 2022


replacement function to flag hospitals that went through consolidation that 1) have no further events or 2) have further change of ownership or acquisition/merger, meaning that a new CCN will appear in later records which I failed to associate with those kinds of post-consolidation hospitals


for index, row in chow_data.iterrows():
    if row['CHOW TYPE TEXT'] == 'Acquisition/Merger':
        -In acquisitions/mergers, the seller's CCN dissolves and the buyer's CCN remains
        ccn_changes[row['CCN - SELLER']] = row['CCN - BUYER']
    elif row['CHOW TYPE TEXT'] == 'Consolidation':
        -In consolidations, both the seller's and buyer's CCNs dissolve, but we don't have a "new CCN"
        -Temporarily map the seller and buyer to "Pending" until further events clarify the new CCN
        ccn_changes[row['CCN - SELLER']] = "Pending"
        ccn_changes[row['CCN - BUYER']] = "Pending"

Recursive function to track CCNs and handle consolidations correctly
def get_final_ccn(ccn):
    path = set()  # to avoid loops
    while ccn in ccn_changes:
        if ccn in path:
            -Handle loops, if any
            break
        path.add(ccn)
        -If CCN is marked as "Pending" from consolidation, check if further events resolve it
        if ccn_changes[ccn] == "Pending":
            return "Unresolved"
        ccn = ccn_changes[ccn]
    return ccn
chow_data['FINAL CCN'] = chow_data['CCN - SELLER'].apply(get_final_ccn)


In [ ]:
print(ccn_mapping)  # Check the mapping of seller to buyer CCNs

{'520204': '520136', '520027': '520051', '240132': '240115', '24S059': '24S038', '340186': '340040', '450558': '450229', '050773': '050320', '330088': '330393', '377107': '370091', '500001': '500008', '330201': '330233', '450766': '459816', '330232': '330180', '330003': '330180', '330409': '330180', '330353': '330195', '270087': '270051', '490079': '490075', '362034': '362019', '420057': '420051', '140250': '140048', '442012': '442015', '390197': '390049', '323029': '320005', '330004': '330224', '150064': '150048', '49S079': '49S075', '050616': '050082', '360081': '360112', '36S081': '360112', '36T081': '360112', '360262': '360112', '160122': '160057', '360362': '360134', '390031': '390030', '390052': '390086', '050153': '050038', '050688': '050038', '210013': '210012', '190246': '190125', '450299': '450011', '390236': '390079', '420083': '420007', '190205': '190102', '251303': '250019A', '370032': '370028', '210045': '210019', '190236': '190111', '232032': '232031', '490011': '490017'

In [ ]:
original_unique_ccns = chow_data['CCN - SELLER'].nunique()
final_unique_ccns = chow_data['FINAL CCN'].nunique()
print(f"Number of unique CCNs in 'CCN - SELLER': {original_unique_ccns}")
print(f"Number of unique CCNs in 'FINAL CCN': {final_unique_ccns}")

# see if any values changed
ccn_changes_detected = chow_data[chow_data['CCN - SELLER'] != chow_data['FINAL CCN']]
print(f"Number of CCN changes detected: {ccn_changes_detected.shape[0]}")

Number of unique CCNs in 'CCN - SELLER': 490
Number of unique CCNs in 'FINAL CCN': 480
Number of CCN changes detected: 78


In [ ]:
# Count the number of acquisition/merger events
num_acquisitions_mergers = chow_data[chow_data['CHOW TYPE TEXT'] == 'ACQUISITION/MERGER'].shape[0]

# Check if the total number of final CCN changes matches expected changes
num_ccn_changes = chow_data[chow_data['CCN - SELLER'] != chow_data['FINAL CCN']].shape[0]

print(f"Number of Acquisition/Merger Events: {num_acquisitions_mergers}")
print(f"Number of CCN Changes Detected: {num_ccn_changes}")


Number of Acquisition/Merger Events: 79
Number of CCN Changes Detected: 78


In [ ]:
# so we have 79 AM and 78 changed values
# filter the acquisition/merger events
AM = chow_data[chow_data['CHOW TYPE TEXT'] == 'ACQUISITION/MERGER']

# check for cases where the seller and buyer have the same CCN
same_ccn_events = AM[AM['CCN - SELLER'] == AM ['CCN - BUYER']]

print(same_ccn_events)

    ENROLLMENT ID - BUYER ENROLLMENT STATE - BUYER PROVIDER TYPE CODE - BUYER  \
130       O20170515002096                       SC                      00-09   

     PROVIDER TYPE TEXT - BUYER  NPI - BUYER MULTIPLE NPI FLAG - BUYER  \
130  PART A PROVIDER - HOSPITAL   1831638238                         N   

    CCN - BUYER  ASSOCIATE ID - BUYER  \
130      422006            2567740657   

                             ORGANIZATION NAME - BUYER  \
130  CONTINUECARE HOSPITAL AT PALMETTO HEALTH BAPTI...   

                       DOING BUSINESS AS NAME - BUYER  ...  \
130  CONTINUECARE HOSPITAL AT PALMETTO HEALTH BAPTIST  ...   

    PROVIDER TYPE CODE - SELLER PROVIDER TYPE TEXT - SELLER NPI - SELLER  \
130                       00-09  PART A PROVIDER - HOSPITAL   1104863331   

    MULTIPLE NPI FLAG - SELLER CCN - SELLER ASSOCIATE ID - SELLER  \
130                          N       422006            5799800033   

                  ORGANIZATION NAME - SELLER  DOING BUSINESS AS NAME - 

In [ ]:
# check for missing CCNs in either 'CCN - SELLER' or 'CCN - BUYER'
missing_ccns = chow_data[chow_data['CCN - SELLER'].isnull() | chow_data['CCN - BUYER'].isnull()]

print(missing_ccns)

Empty DataFrame
Columns: [ENROLLMENT ID - BUYER, ENROLLMENT STATE - BUYER, PROVIDER TYPE CODE - BUYER, PROVIDER TYPE TEXT - BUYER, NPI - BUYER, MULTIPLE NPI FLAG - BUYER, CCN - BUYER, ASSOCIATE ID - BUYER, ORGANIZATION NAME - BUYER, DOING BUSINESS AS NAME - BUYER, CHOW TYPE CODE, CHOW TYPE TEXT, EFFECTIVE DATE, ENROLLMENT ID - SELLER, ENROLLMENT STATE - SELLER, PROVIDER TYPE CODE - SELLER, PROVIDER TYPE TEXT - SELLER, NPI - SELLER, MULTIPLE NPI FLAG - SELLER, CCN - SELLER, ASSOCIATE ID - SELLER, ORGANIZATION NAME - SELLER, DOING BUSINESS AS NAME - SELLER, Year, FINAL CCN]
Index: []

[0 rows x 25 columns]


In [ ]:
compendium_data = pd.read_csv('/hospital2022.csv',encoding='ISO-8859-1')
# drop rows where the 'CCN' value is missing/non CMS providers
compendium_data.dropna(subset=['ccn'], inplace=True)
# ensure string format
compendium_data['ccn'] = compendium_data['ccn'].astype(str)

In [ ]:
# merge using the FINAL CCN to align with the latest CCN
final_data = pd.merge(compendium_data, chow_data, left_on='ccn', right_on='FINAL CCN', how='left')


In [ ]:
# create an empty dataframe with years *2016-2022* for all hospitals in compendium_data (list from 2022)
# to have a row for each hopsital in each year regardless of events happening or not
import pandas as pd

# Create an empty dataframe with years 2016-2022 for all hospitals in compendium_data (list from 2022)
years = range(2016, 2023)
all_hospitals = pd.DataFrame([(ccn, year) for ccn in compendium_data['ccn'] for year in years], columns=['ccn', 'Year'])

# 'static' hospital details (name, address, etc.) merged with all year rows
static_columns = ['ccn', 'hospital_name', 'hospital_state', 'hospital_city', 'hospital_street', 'hospital_zip', 'compendium_hospital_id']

# other characteristics specific to 2022 merged only with 2022 rows
characteristics_columns = [col for col in compendium_data.columns if col not in static_columns]

# merge static hospital details with all years
hospital_data_all_years = pd.merge(all_hospitals, compendium_data[static_columns], on='ccn', how='left')

# filter only 2022 rows for annual characteristics (correcting the list concatenation)
hospital_data_2022 = all_hospitals[all_hospitals['Year'] == 2022].merge(compendium_data[['ccn'] + characteristics_columns], on='ccn', how='left')

# number of each event type per year per hospital
event_counts = chow_data.groupby(['FINAL CCN', 'Year', 'CHOW TYPE TEXT']).size().unstack(fill_value=0)

# define column names, including a column for 'CONSOLIDATION' filled with zeros
event_counts.columns = ['ACQUISITION/MERGER', 'CHANGE OF OWNERSHIP']
event_counts['CONSOLIDATION'] = 0

# reindex event_counts to ensure it includes all hospitals and all years (filling missing values with 0)
event_counts = event_counts.reindex(pd.MultiIndex.from_product([compendium_data['ccn'], years], names=['FINAL CCN', 'Year']), fill_value=0)

# reset the index to merge event counts
event_counts = event_counts.reset_index()

# merge event counts with the static hospital data
hospital_year_data = pd.merge(hospital_data_all_years, event_counts, left_on=['ccn', 'Year'], right_on=['FINAL CCN', 'Year'], how='left')

# merge characteristics only for the 2022 row
hospital_year_data = pd.merge(hospital_year_data, hospital_data_2022, on=['ccn', 'Year'], how='left')

hospital_year_data.head(10)


,ccn,Year,hospital_name,hospital_state,hospital_city,hospital_street,hospital_zip,compendium_hospital_id,FINAL CCN,ACQUISITION/MERGER,...,hos_children,hos_majteach,hos_vmajteach,hos_teachint,hos_highdpp,hos_ucburden,hos_highuc,hos_ownership,hos_net_revenue,hos_total_revenue
0,390163,2016,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,390163,2017,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,390163,2018,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,390163,2019,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,390163,2020,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,390163,2021,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,390163,2022,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,CHSP00000003,390163,0,...,0.0,0.0,0.0,0.0,0.0,0.037058,0.0,1.0,115554498.0,399334161.0
7,050226,2016,Anaheim Regional Medical Center,CA,Anaheim,1111 West La Palma Avenue,92801,CHSP00000004,050226,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,050226,2017,Anaheim Regional Medical Center,CA,Anaheim,1111 West La Palma Avenue,92801,CHSP00000004,050226,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,050226,2018,Anaheim Regional Medical Center,CA,Anaheim,1111 West La Palma Avenue,92801,CHSP00000004,050226,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
check=hospital_year_data['ACQUISITION/MERGER'].value_counts()
print(check)

ACQUISITION/MERGER
0    46371
1       49
2        3
4        1
Name: count, dtype: int64


In [ ]:
# format 1 export
hospital_year_data.to_csv('hospital_year.csv', index=False)

In [ ]:
# merge compendium data with the CHOW data on FINAL CCN (hospital CCN after CHOW event)
final_data = pd.merge(compendium_data, chow_data, left_on='ccn', right_on='FINAL CCN', how='left')

# create a status column: 1 if there was a CHOW event, 0 if there was no event
final_data['CHOW Event Status'] = final_data['CHOW TYPE TEXT'].apply(lambda x: 1 if pd.notnull(x) else 0)

# select relevant columns for hospital and event details
hospital_event_data = final_data[['ccn', 'hospital_name', 'hospital_state', 'hospital_city','hospital_street','hospital_zip', 'CHOW TYPE TEXT', 'EFFECTIVE DATE','Year', 'CCN - BUYER', 'CCN - SELLER', 'ORGANIZATION NAME - BUYER', 'ORGANIZATION NAME - SELLER', 'CHOW Event Status']]

# fill CHOW event status with 0 for hospitals with no events
hospital_event_data['CHOW Event Status'].fillna(0, inplace=True)

hospital_event_data.head()

<ipython-input-18-a87fa2de8662>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hospital_event_data['CHOW Event Status'].fillna(0, inplace=True)
<ipython-input-18-a87fa2de8662>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_event_data['CHOW Event Status'].fillna(0, inplace=True)


,ccn,hospital_name,hospital_state,hospital_city,hospital_street,hospital_zip,CHOW TYPE TEXT,EFFECTIVE DATE,Year,CCN - BUYER,CCN - SELLER,ORGANIZATION NAME - BUYER,ORGANIZATION NAME - SELLER,CHOW Event Status
0,390163,Armstrong County Memorial,PA,Kittanning,One Nolte Drive,16201,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0
1,050226,Anaheim Regional Medical Center,CA,Anaheim,1111 West La Palma Avenue,92801,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0
2,292007,Las Vegas Amg Specialty Hospital,NV,Las Vegas,4015 Mcleod Drive,89121,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0
3,152025,Central Indiana-Amg Specialty Hospit,IN,Muncie,"2401 West University Avenue, 8th Floor",47303,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0
4,322003,Albuquerque - Amg Specialty Hospital,NM,Albuquerque,"5400 Gibson Boulevard Se, 3rd Floor",87108,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# format 2 export
hospital_event_data.to_csv('hospital_event.csv', index=False)